배송지연 여부 예측모델

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


df = pd.read_csv("Sales_Data05.csv", encoding="cp949") 

# 날짜 데이터 변환
df['배송시작일'] = pd.to_datetime(df['배송시작일'])
df['배송완료일'] = pd.to_datetime(df['배송완료일'])

# 배송 지연 여부  
df['배송지연'] = ((df['배송완료일'] - df['배송시작일']).dt.days >= 2).astype(int)

# 필요한 컬럼
df = df[['구매수량', '구매금액', '사용 적립금', '사용 포인트 네이버', '배송지연']].dropna()

# 특성, 타겟 설정
X = df.drop(columns=['배송지연'])
y = df['배송지연']

# 데이터 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 학습
model = RandomForestClassifier()
model.fit(X_train, y_train)

In [73]:
# 예측 및 정확도 평가
y_pred = model.predict(X_test)
print("정확도:", accuracy_score(y_test, y_pred))

정확도: 0.8522559738967094


베송지연 경험 고객 비율

In [84]:
import pandas as pd

# 데이터 불러오기
df_sales = pd.read_csv("Sales_Data05.csv", encoding="cp949")

# 날짜 변환 
df_sales['배송시작일'] = pd.to_datetime(df_sales['배송시작일'], errors='coerce')
df_sales['배송완료일'] = pd.to_datetime(df_sales['배송완료일'], errors='coerce')

# 배송 지연 여부 계산 
df_sales['배송지연'] = (df_sales['배송완료일'] - df_sales['배송시작일']).dt.days >= 2

# 배송 지연 경험 고객 비율 계산
delayed_customers_ratio = df_sales['배송지연'].mean() * 100

# 결과 출력
print(f"배송 지연을 경험한 고객 비율: {delayed_customers_ratio:.2f}%")

배송 지연을 경험한 고객 비율: 14.53%


배송지연과 재구매율

In [86]:
import pandas as pd

# 데이터 불러오기
df_sales = pd.read_csv("Sales_Data05.csv", encoding="cp949")
# 날짜 변환
df_sales[['배송시작일', '배송완료일']] = df_sales[['배송시작일', '배송완료일']].apply(pd.to_datetime)

# 배송 지연 여부 (배송완료일 - 배송시작일이 2일 이상)
df_sales['배송지연'] = (df_sales['배송완료일'] - df_sales['배송시작일']).dt.days >= 2

# 배송 지연을 경험한 고객 리스트
delayed_customers = df_sales[df_sales['배송지연']]['회원번호'].unique()

# 재구매 여부 (회원번호가 여러 번 등장하면 재구매)
df_sales['재구매'] = df_sales.duplicated(subset=['회원번호'], keep=False).astype(int)

# 배송 지연 고객들의 재구매율 계산
delayed_repurchase_rate = df_sales[df_sales['회원번호'].isin(delayed_customers)]['재구매'].mean()

# 결과 출력
print(f'배송 지연을 경험한 고객들의 재구매율: {delayed_repurchase_rate:.2%}')

배송 지연을 경험한 고객들의 재구매율: 99.98%


배송지연과 고객이탈율

In [62]:
df_member = pd.read_csv("Member_Data.csv", encoding="cp949")

# 고객별 최근 구매일 계산 (가장 최근 주문일시)
latest_purchase = df_sales.groupby('회원번호')['주문일시'].max().reset_index()
latest_purchase.rename(columns={'주문일시': '최근구매일'}, inplace=True)

In [63]:
# `merge` 사용하여 회원 데이터에 최근 구매일 추가
df_member = df_member.merge(latest_purchase, on='회원번호', how='left')

def check_ghost_rate(n):
    cutoff_date = df_sales['주문일시'].max() - pd.DateOffset(days=n)
    ghost_rate = (df_member['최근구매일'] < cutoff_date).mean() * 100
    print(f"최근 {n}일간 주문 없는 유령 고객 비율: {ghost_rate:.2f}%")

In [64]:
def check_churn_rate():
    # 배송 지연 여부별 고객 이탈률 계산
    # 회원 정보에 고객 이탈 여부 추가 (구매 내역이 없거나 탈퇴한 회원)
    df_member['고객이탈'] = df_member['최근구매일'].isna() | (df_member['회원상태'] == '탈퇴')

    # df_sales`와 `df_member` 병합하여 고객 이탈 여부 포함
    df_sales_temp = df_sales.merge(df_member[['회원번호', '고객이탈']], on='회원번호', how='left')

    # 배송 지연 여부별 고객 이탈률 계산 및 출력
    print(df_sales_temp.pivot_table(index='배송지연', values='고객이탈', aggfunc='mean'))

In [75]:
check_ghost_rate(n=60)
check_churn_rate()

최근 60일간 주문 없는 유령 고객 비율: 37.26%
           고객이탈
배송지연           
False  0.011090
True   0.011003


RFM기반 클러스터별 배송지연비율

In [81]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

df_rfm = pd.read_csv("RFM_Score.csv", encoding="utf-8")

# 날짜 변환 
df_sales['배송시작일'] = pd.to_datetime(df_sales['배송시작일'], errors='coerce', dayfirst=True)
df_sales['배송완료일'] = pd.to_datetime(df_sales['배송완료일'], errors='coerce', dayfirst=True)

# 배송 지연 여부 
df_sales['배송지연'] = (df_sales['배송완료일'] - df_sales['배송시작일']).dt.days >= 2

# 고객별 배송 지연 경험 비율 계산
customer_delay_rate = df_sales.groupby('회원번호')['배송지연'].mean().reset_index()
customer_delay_rate.rename(columns={'배송지연': '배송지연비율'}, inplace=True)

# RFM 데이터 정규화
rfm_features = df_rfm[['Recency', 'Frequency', 'Monetary']]
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_features)

# K-Means 클러스터링 (K=4)
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_rfm['클러스터'] = kmeans.fit_predict(rfm_scaled)

# RFM 데이터와 배송 지연 데이터 병합
df_rfm = df_rfm.merge(customer_delay_rate, on='회원번호', how='left')

# 클러스터별 배송 지연 경험 비율 계산
cluster_delay_rate = df_rfm.groupby("클러스터")["배송지연비율"].mean().reset_index()

print(cluster_delay_rate)

   클러스터    배송지연비율
0     0  0.140700
1     1  0.146076
2     2  0.145162
3     3  0.145034
